<a href="https://colab.research.google.com/github/weihaolun/Twitter-Sentiment-Analysis/blob/main/Database/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:2 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic-backports InRelease [74.6 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Get:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:6 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:7 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:8 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Get:9 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:10 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:11 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Hit:13 https://developer.download.n

In [2]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

--2021-11-15 05:52:05--  https://jdbc.postgresql.org/download/postgresql-42.2.16.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1002883 (979K) [application/java-archive]
Saving to: ‘postgresql-42.2.16.jar’

postgresql-42.2.16. 100%[===================>] 979.38K  1.69MB/s    in 0.6s    

2021-11-15 05:52:06 (1.69 MB/s) - ‘postgresql-42.2.16.jar’ saved [1002883/1002883]



In [3]:
# Create spark session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("final-project").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

### Load Amazon Data into Spark DataFrames

In [4]:
from pyspark import SparkFiles
# positive tweets 
url = "https://sjpv94twitter.s3.us-west-1.amazonaws.com/positive.csv"
spark.sparkContext.addFile(url)
positive_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("positive.csv"), sep=",", header=True, inferSchema=True)
positive_df.show()

+---+--------+--------------------+
|_c0|attitude|              tweets|
+---+--------+--------------------+
|  5|positive|Tim Tebow may be ...|
|  6|positive|@mariakaykay aga ...|
|  8|positive|Lunch from my new...|
|  9|positive|SNC Halloween Pr....|
| 14|positive|Louis inspired ou...|
| 17|positive|"""I love the ban...|
| 18|positive|#Repost Chris Bos...|
| 20|positive|"""Gary Ablett wi...|
| 23|positive|Lana Del Rey at H...|
| 26|positive|ABC has @jaketapp...|
| 27|positive|Here in the Phili...|
| 28|positive|Tonight Dr. Terri...|
| 32|positive|@solz_b He\u2019s...|
| 33|positive|Patriots Extend L...|
| 34|positive|@KevOrf_5 Yeah I ...|
| 38|positive|Pretty Little Lia...|
| 39|positive|@MonicaGonzo Texa...|
| 41|positive|GREAT GAME GIRLS!...|
| 46|positive|#7FactsAboutMyBes...|
| 48|positive|waking up to a Ni...|
+---+--------+--------------------+
only showing top 20 rows



In [5]:
# negative tweets 
url = "https://sjpv94twitter.s3.us-west-1.amazonaws.com/negative.csv"
spark.sparkContext.addFile(url)
negative_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("negative.csv"), sep=",", header=True, inferSchema=True)
negative_df.show()

+---+--------+--------------------+
|_c0|attitude|              tweets|
+---+--------+--------------------+
|  2|negative|@francesco_con40 ...|
| 10|negative|@jacquelinemegan ...|
| 19|negative|@prodnose is this...|
| 21|negative|Gold edges down a...|
| 25|negative|Monday before I l...|
| 31|negative|\u2019Love-cheat\...|
| 44|negative|Cardinal men take...|
| 51|negative|@Duffy_Louise Noo...|
| 53|negative|I believe the sno...|
| 55|negative|Well if no ones g...|
| 57|negative|Watching Contraba...|
| 58|negative|@JoshNorris @Roto...|
| 61|negative|Steal by Chalmers...|
| 76|negative|The Rick Santorum...|
| 85|negative|@DannyB618 Sure a...|
| 86|negative|@RichardGordon48 ...|
| 94|negative|#pause I bet the ...|
|104|negative|Mitt Romney false...|
|110|negative|Firework just cam...|
|111|negative|@TatiCuteAss you ...|
+---+--------+--------------------+
only showing top 20 rows



In [6]:
# neutral tweets 
url = "https://sjpv94twitter.s3.us-west-1.amazonaws.com/neutral.csv"
spark.sparkContext.addFile(url)
neutral_df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("neutral.csv"), sep=",", header=True, inferSchema=True)
neutral_df.show()

+---+--------+--------------------+
|_c0|attitude|              tweets|
+---+--------+--------------------+
|  0| neutral|Won the match #ge...|
|  1| neutral|Some areas of New...|
|  3| neutral|#Thailand Washing...|
|  4| neutral|Did y\u2019all he...|
|  7| neutral|Tina Fey & Amy Po...|
| 11| neutral|Manchester United...|
| 12| neutral|Going to a bulls ...|
| 13| neutral|Any Toon Fans wit...|
| 15| neutral|going to bed now....|
| 16| neutral|@_Nenaah oh cause...|
| 22| neutral|.@NUMensSoccer: A...|
| 24| neutral|Shaw wouldn\u2019...|
| 29| neutral|Man\u002c bye. I ...|
| 30| neutral|@thaalitaa410 won...|
| 35| neutral|I may exit off tw...|
| 36| neutral|Indiana 1\u002c N...|
| 37| neutral|"""VH1\u2019s \""...|
| 40| neutral|If you are in Van...|
| 42| neutral|Jesiah of course ...|
| 43| neutral|@gleekyspnluver @...|
+---+--------+--------------------+
only showing top 20 rows



In [8]:
# api tweets 
url = "https://sjpv94twitter.s3.us-west-1.amazonaws.com/cleaned_scored_tweets.json"
spark.sparkContext.addFile(url)
api_tweets_df = spark.read.option("encoding", "UTF-8").json(SparkFiles.get("cleaned_scored_tweets.json"))
api_tweets_df.show()

+-------------+------------+-----+-----+--------------------+--------------------+-------+
|   created_at|created_date|hours|score|                text|               tweet|weekday|
+-------------+------------+-----+-----+--------------------+--------------------+-------+
|1635728396000|  2021-11-01|    0|    0|[car, hardware, n...|@lj3252 @Teslarat...| Monday|
|1635728393000|  2021-11-01|    0|    1|[hi, trying, star...|@elonmusk @garycr...| Monday|
|1635728386000|  2021-11-01|    0|    1|[real, interest, ...|@AgnesOfTheDogs @...| Monday|
|1635728381000|  2021-11-01|    0|    1|[hi, trying, star...|@elonmusk @garycr...| Monday|
|1635728377000|  2021-11-01|    0|    0|[elon, musk, say,...|Elon Musk says he...| Monday|
|1635728376000|  2021-11-01|    0|    0|[doge, dogearmy, ...|#doge #dogearmy #...| Monday|
|1635728368000|  2021-11-01|    0|    1|[according, nhtsa...|According to the ...| Monday|
|1635728366000|  2021-11-01|    0|    0|[exactly, person,...|@llIIIllIIIIlll @...| Monday|

In [9]:
# api count
url = "https://sjpv94twitter.s3.us-west-1.amazonaws.com/tweets_count.json"
spark.sparkContext.addFile(url)
api_count_df = spark.read.option("encoding", "UTF-8").json(SparkFiles.get("tweets_count.json"))
api_count_df.show()

+------------+--------------------+--------------------+-----------+---------+
|created_date|            end_time|          start_time|tweet_count|  weekday|
+------------+--------------------+--------------------+-----------+---------+
|  2021-11-01|2021-11-02T00:00:...|2021-11-01T00:00:...|     138498|   Monday|
|  2021-11-02|2021-11-03T00:00:...|2021-11-02T00:00:...|      88621|  Tuesday|
|  2021-11-03|2021-11-04T00:00:...|2021-11-03T00:00:...|      53148|Wednesday|
|  2021-11-04|2021-11-05T00:00:...|2021-11-04T00:00:...|      54606| Thursday|
|  2021-11-05|2021-11-06T00:00:...|2021-11-05T00:00:...|      47726|   Friday|
|  2021-11-06|2021-11-07T00:00:...|2021-11-06T00:00:...|      64812| Saturday|
|  2021-11-07|2021-11-08T00:00:...|2021-11-07T00:00:...|      78984|   Sunday|
|  2021-11-08|2021-11-09T00:00:...|2021-11-08T00:00:...|      74188|   Monday|
|  2021-11-09|2021-11-10T00:00:...|2021-11-09T00:00:...|      51366|  Tuesday|
|  2021-11-10|2021-11-11T00:00:...|2021-11-10T00:00:

### Create DataFrames to match tables in pgAdmin

In [10]:
# combine positive, negative, neutral tweets to match tables in Postgres
initial_tweets = positive_df.unionByName(negative_df)
initial_tweets = initial_tweets.unionByName(neutral_df)
initial_tweets = initial_tweets.drop(initial_tweets._c0)
initial_tweets.show()

+--------+--------------------+
|attitude|              tweets|
+--------+--------------------+
|positive|Tim Tebow may be ...|
|positive|@mariakaykay aga ...|
|positive|Lunch from my new...|
|positive|SNC Halloween Pr....|
|positive|Louis inspired ou...|
|positive|"""I love the ban...|
|positive|#Repost Chris Bos...|
|positive|"""Gary Ablett wi...|
|positive|Lana Del Rey at H...|
|positive|ABC has @jaketapp...|
|positive|Here in the Phili...|
|positive|Tonight Dr. Terri...|
|positive|@solz_b He\u2019s...|
|positive|Patriots Extend L...|
|positive|@KevOrf_5 Yeah I ...|
|positive|Pretty Little Lia...|
|positive|@MonicaGonzo Texa...|
|positive|GREAT GAME GIRLS!...|
|positive|#7FactsAboutMyBes...|
|positive|waking up to a Ni...|
+--------+--------------------+
only showing top 20 rows



In [12]:
# rearrange columns
from pyspark.sql.functions import col
from pyspark.sql.functions import to_date
api_tweets_df = api_tweets_df.select(["created_at", to_date("created_date", 'yyyy-MM-dd').alias("created_date"), "weekday",\
                                      col("hours").cast("int"), "tweet", "text", "score"])
api_tweets_df.show()

+-------------+------------+-------+-----+--------------------+--------------------+-----+
|   created_at|created_date|weekday|hours|               tweet|                text|score|
+-------------+------------+-------+-----+--------------------+--------------------+-----+
|1635728396000|  2021-11-01| Monday|    0|@lj3252 @Teslarat...|[car, hardware, n...|    0|
|1635728393000|  2021-11-01| Monday|    0|@elonmusk @garycr...|[hi, trying, star...|    1|
|1635728386000|  2021-11-01| Monday|    0|@AgnesOfTheDogs @...|[real, interest, ...|    1|
|1635728381000|  2021-11-01| Monday|    0|@elonmusk @garycr...|[hi, trying, star...|    1|
|1635728377000|  2021-11-01| Monday|    0|Elon Musk says he...|[elon, musk, say,...|    0|
|1635728376000|  2021-11-01| Monday|    0|#doge #dogearmy #...|[doge, dogearmy, ...|    0|
|1635728368000|  2021-11-01| Monday|    0|According to the ...|[according, nhtsa...|    1|
|1635728366000|  2021-11-01| Monday|    0|@llIIIllIIIIlll @...|[exactly, person,...|    0|

In [14]:
# rearrange columns
api_count_df = api_count_df.select(["end_time","start_time",to_date("created_date", 'yyyy-MM-dd').alias("created_date"),"weekday","tweet_count"])
api_count_df.show()

+--------------------+--------------------+------------+---------+-----------+
|            end_time|          start_time|created_date|  weekday|tweet_count|
+--------------------+--------------------+------------+---------+-----------+
|2021-11-02T00:00:...|2021-11-01T00:00:...|  2021-11-01|   Monday|     138498|
|2021-11-03T00:00:...|2021-11-02T00:00:...|  2021-11-02|  Tuesday|      88621|
|2021-11-04T00:00:...|2021-11-03T00:00:...|  2021-11-03|Wednesday|      53148|
|2021-11-05T00:00:...|2021-11-04T00:00:...|  2021-11-04| Thursday|      54606|
|2021-11-06T00:00:...|2021-11-05T00:00:...|  2021-11-05|   Friday|      47726|
|2021-11-07T00:00:...|2021-11-06T00:00:...|  2021-11-06| Saturday|      64812|
|2021-11-08T00:00:...|2021-11-07T00:00:...|  2021-11-07|   Sunday|      78984|
|2021-11-09T00:00:...|2021-11-08T00:00:...|  2021-11-08|   Monday|      74188|
|2021-11-10T00:00:...|2021-11-09T00:00:...|  2021-11-09|  Tuesday|      51366|
|2021-11-11T00:00:...|2021-11-10T00:00:...|  2021-11

### Connect to the AWS RDS instance and write each DataFrame to its table. 

In [15]:
# Configure settings for RDS
from getpass import getpass
password = getpass('Enter database password')

mode = "append"
jdbc_url="jdbc:postgresql://<connection string>/<dbname>"
config = {"user":"postgres", 
          "password": password, 
          "driver":"org.postgresql.Driver"}

# Deleted connection string, and db name
# For the purpose of this project,
# connection string used was 'twitterproject.cgrabvwy3u7o.us-west-1.rds.amazonaws.com:5432'
# db name used was 'twitterproject'
# Database will be deleted after project submission

Enter database password··········


In [16]:
# Write initial_tweets to table in RDS
initial_tweets.write.jdbc(url=jdbc_url, table='initial_tweets_table', mode=mode, properties=config)

In [17]:
# Write api_tweets_df to table in RDS
api_tweets_df.write.jdbc(url=jdbc_url, table='api_tweets_table', mode=mode, properties=config)

In [18]:
# Write api_count_df to table in RDS
api_count_df.write.jdbc(url=jdbc_url, table='api_count_table', mode=mode, properties=config)